In [ ]:
%pip install numpy
%pip install pulp

     |████████████████████████████████| 40.6MB 100kB/s 


In [ ]:
import numpy as np
import pulp #Importar a biblioteca de modelagem
import random #Importações do Python
import time

 # **Modelagem Matemática**

A Modelagem matemática está descrita no documento apresentado. A função que define a modelagem é dada abaixo:

In [ ]:
def BPP_model(items):
  itemCount = len(items)

  # Número Máximo de Bins
  maxBins = itemCount

  # Volume dos Bins
  binCapacity = 1000

  # Definições das Variáveis de Decisão
  y = pulp.LpVariable.dicts('BinUsado', range(maxBins),
                            lowBound = 0,
                            upBound = 1,
                            cat = LpInteger)
  possible_ItemInBin = [(itemTuple[0], binNum) for itemTuple in items
                                            for binNum in range(maxBins)]
  x = pulp.LpVariable.dicts('ItemNoBin', possible_ItemInBin,
                            lowBound = 0,
                            upBound = 1,
                            cat = LpInteger)


  # Inicializando o problema.
  prob = LpProblem("Trabalho Computacional 1 - Bin Packing Problem", LpMinimize)

  # Adicionando a função objetivo.
  prob += lpSum([y[i] for i in range(maxBins)]), "Objetivo: Minimizar a quantidade de bins"

  # Primeira Restrição: Para todo item, a soma dos bins que ele aparece deve ser 1.
  for j in items:
    prob += lpSum([x[(j[0], i)] for i in range(maxBins)]) == 1, ("Cada item só pode estar em 1 Bin -- " + str(j[0]))

  # Segunda Restrição:Para todo Bin, o número do volume dos itens dentro do bin deve ser menor que o volume do bin.
  for i in range(maxBins):
    prob += lpSum([items[j][1] * x[(items[j][0], i)] for j in range(itemCount)]) <= binCapacity*y[i], ("A soma dos volumes dos objetos deve ser menor que a do Bin -- " + str(i))

  # Escrevendo o modelo num arquivo .lp
  prob.writeLP("BinPack.lp")

  # Começaremos a contar o tempo da solução
  start_time = time.time()

  # Resolvendo o problema
  prob.solve(use_mps=False)
  
  print("Bins utilizados: " + str(sum(([y[i].value() for i in range(maxBins)]))))
  print("Resolvido em %s segundos." % (time.time() - start_time))

Não foi testado o modelo matemático para as instâncias abaixo porque como o BPP é NP-Hard, para uma quantidade de 200 objetos, o problema segue um nivel fatorial, portanto não pode ser resolvido em tempo polinomial.

# **Métodos das Heurísticas**

Para resolver o BPP, foi instaurado que utilizariamos uma função definida como a objetivo (sendo essa a função minimiza) e outras duas heurísticas combinadas com a função objetivo.









In [ ]:
def func_minimiza(qnt_obj, Volume_obj, Volume_rec): 
    
    for j in range(qnt_obj):  #Verifica se os objetos cabem no recipiente.
      if Volume_obj[j]>Volume_rec:
        print('\n')
        print(f'Um dos objetos não cabem no recipiente de volume {Volume_rec}')
        return qnt_recipientes  
    
      Aux_volume_rec = Volume_rec
      qnt_recipientes = 1 
      for i in range(qnt_obj): 
        if Aux_volume_rec >= Volume_obj[i]: 
          Aux_volume_rec = Aux_volume_rec - Volume_obj[i] 
        else: 
          qnt_recipientes += 1
          Aux_volume_rec = Volume_rec - Volume_obj[i] 
    return qnt_recipientes

A **primeira heurística** tem por seu interesse tomar um vetor de n objetos com volumes distintos e gerar, aleatoriamente, para a função minimiza. Ou seja, ele bagunça o vetor de objetos para encontrar uma boa solução.

In [ ]:
def OTM_shuffle(qnt_entradas, Volume_dos_obj, V):
  k=0
  melhor_sol = 1000000

  ini = time.time()
  while k<10:
    k +=1
    random.shuffle(Volume_dos_obj) #Embaralha o vetor
    sol = func_minimiza(qnt_entradas, Volume_dos_obj, V)
    #print(f'{Volume_dos_obj}: {sol}') Imprime as soluções geradas
    if sol < melhor_sol:
      melhor_sol = sol
  fim = time.time()

  print(f'A melhor solução encontrada foi {melhor_sol}')
  print(f'Tempo de execução com k = {k} : {fim-ini} segundos')

A **segunda heurística** tem por seu interesse ordenar esse vetor decrescentemente e combinar seus valores na função minimiza para encontrar a boa solução.

In [ ]:
def OTM_decresce(qnt_entradas, Volume_dos_obj, V):
  ini = time.time()
  Volume_dos_obj.sort(reverse=True)
  sol = func_minimiza(qnt_entradas, Volume_dos_obj, V)
  #print(f'{Volume_dos_obj}: {sol}') Imprime as soluções geradas

  fim = time.time()
  print(f'A melhor solução encontrada foi {sol}')
  print(f'Tempo de execução: {fim-ini} segundos')

# **Primeiro exemplo**

Testamos o código para um vetor finito de 14 elementos, para depois aplicarmos nas instâncias de 160 a 200 elementos.

In [ ]:
Volume_dos_obj =[1,5,4,7,2,3,8,1,5,4,7,2,3,8]
qnt_entradas=len(Volume_dos_obj)
V=10

OTM_shuffle(qnt_entradas, Volume_dos_obj, V)

In [ ]:
Volume_dos_obj =[1,5,4,7,2,3,8,1,5,4,7,2,3,8,]
qnt_entradas=len(Volume_dos_obj)
V=10

OTM_decresce(qnt_entradas, Volume_dos_obj, V)

A melhor solução encontrada foi 8
Tempo de execução: 4.792213439941406e-05 segundos


# **Exemplos de Instâncias:**

As instâncias são dadas pelo J.E. Schoenfield in Fast, exact solution of open bin packing problems without linear programming. Technical report, US Army Space and Missile Defense Command, Huntsville, Alabama, USA, 2002.

Eis alguns exemplos de instâncias abaixo dada na listas:

In [ ]:
#Hard BPP 40
Inst_1 = [789,787,786,781,777,774,773,770,763,763,757,753,748,727,726,716,707,703,700,695,681,676,676,675,675,664,661,654,654,649,645,639,626,620,613,612,605,605,601,689,586,577,562,561,548,536,523,511,
500,498,98,493,488,484,484,483,482,478,464,461,461,458,438,436,434,431,420,419,416,415,414,413,410,400,393,385,373,373,372,369,351,351,342,321,320,320,317,313,307,301,300,299,288,275,270,267,260,257,256,253,253,248,
237,234,227,216,214,208,202,192,191,188,173,169,164,160,155,151,145,140,131,130,127,124,117,116,108,108,107,106,97,94,92,86,80,77,76,75,73,73,70,67,62,59,54,53,50,50,49,48,46,41,40,40,32,29,28,27,24,19]

#Hard BPP 60
Inst_2 = [696,690,687,681,679,679,678,677,676,675,665,658,651,651,650,645,645,644,642,640,640,636,635,631,630,629,616,610,609,607,598,596,595,589,579,578,574,571,570,559,556,555,553,551,545,536,534,530,
530,522,514,510,503,501,496,493,493,491,488,487,482,476,471,470,465,460,456,455,454,454,451,444,442,440,439,436,435,434,428,427,424,420,408,403,403,394,391,385,382,377,377,374,365,357,357,355,341,329,328,328,327,318,
312,300,297,294,291,290,284,281,277,277,265,257,248,245,244,242,236,230,227,225,201,196,191,172,165,164,160,158,157,154,146,145,145,141,139,136,128,126,123,123,120,116,111,109,108,105,105,101,90,88,88,85,80,77,72,
65,64,53]

#Hard BPP 144
Inst_3 = [699,699,696,694,693,692,691,690,689,688,682,681,679,679,673,673,671,664,662,661,655,652,649,646,640,638,618,617,602,590,587,586,586,584,582,568,567,563,560,558,553,548,539,539,536,536,532,532,
529,526,526,522,521,517,516,514,513,510,507,505,502,498,495,493,491,485,482,477,475,474,465,456,454,453,450,442,441,440,439,437,437,434,424,422,419,415,407,404,401,399,395,395,393,391,386,384,382,381,368,354,353,348,
44,343,343,343,336,336,328,328,324,324,321,319,319,304,302,301,300,296,295,284,281,280,279,275,275,271,267,260,257,253,248,245,243,239,238,238,237,233,226,225,222,222,218,217,216,216,212,211,206,204,201,197,189,188,187,
186,181,176,173,167,165,163,158,153,146,146,142,135,127,126,124,119,115,115,114,107,105,104,96,96,94,94,79,61,55,54,44,44,33,31,28,28,18,15,15,12,11]

#Hard BPP 178
Inst_4 = [794,793,790,786,772,764,756,743,742,740,740,736,736,732,724,724,723,713,712,709,704,703,700,699,692,691,691,678,677,673,667,662,662,659,658,646,644,640,637,635,633,629,628,625,622,617,613,612,608,
608,605,602,595,595,583,580,572,568,558,541,535,531,523,521,519,518,518,518,517,517,515,512,509,506,506,504,494,487,487,485,482,476,471,469,468,466,462,459,447,446,443,436,431,423,423,419,416,415,408,407,400,399,396,395,
387,371,370,370,369,365,363,360,355,354,347,346,345,337,336,334,328,321,313,312,292,278,278,276,275,266,257,254,252,250,250,249,246,242,240,233,233,232,226,223,221,220,214,213,212,210,209,208,196,196,189,185,185,184,181,
179,174,171,170,168,166,164,157,147,146,141,133,123,120,119,116,116,104,103,102,99,94,93,72,66,61,61,61,59,55,51,50,45,44,41,36,21,9,7,4,1]


#**Calculando Instâncias:**

Para ambas heurísticas, há uma melhor solução na primeira heurística. Para a análise de complexidade do método, temos que ambas estão na mesma quantidade de tempo, oscilando entre a primeira e a segunda heurística.

In [ ]:
with open('Hard28_BPP13.txt', 'r') as f:
    results = [int(line) for line in f.readlines()]
    

qnt_entradas = results[0]
results.remove(results[0])
results.remove(1000)
Inst_1 = results

OTM_shuffle(qnt_entradas, Inst_1, 1000)
print('\n')
OTM_decresce(qnt_entradas, Inst_1, 1000)

A melhor solução encontrada foi 88
Tempo de execução com k = 10 : 0.03365731239318848 segundos


A melhor solução encontrada foi 96
Tempo de execução: 0.004384040832519531 segundos


In [ ]:
with open('Hard28_BPP40.txt', 'r') as f:
    results = [int(line) for line in f.readlines()]
    #print(results)

qnt_entradas = results[0]
results.remove(results[0])
results.remove(1000)
#print(results)
Inst_2 = results

OTM_shuffle(qnt_entradas, Inst_2, 1000)
print('\n')
OTM_decresce(qnt_entradas, Inst_2, 1000)

A melhor solução encontrada foi 75
Tempo de execução com k = 10 : 0.02648329734802246 segundos


A melhor solução encontrada foi 79
Tempo de execução: 0.0026025772094726562 segundos


In [ ]:
with open('Hard28_BPP60.txt', 'r') as f:
    results = [int(line) for line in f.readlines()]
    #print(results)

qnt_entradas = results[0]
results.remove(results[0])
results.remove(1000)
#print(results)
Inst_3 = results

OTM_shuffle(qnt_entradas, Inst_3, 1000)
print('\n')
OTM_decresce(qnt_entradas, Inst_3, 1000)

A melhor solução encontrada foi 83
Tempo de execução com k = 10 : 0.0281829833984375 segundos


A melhor solução encontrada foi 88
Tempo de execução: 0.0025620460510253906 segundos


In [ ]:
with open('Hard28_BPP144.txt', 'r') as f:
    results = [int(line) for line in f.readlines()]
    #print(results)

qnt_entradas = results[0]
results.remove(results[0])
results.remove(1000)
#print(results)
Inst_4 = results

OTM_shuffle(qnt_entradas, Inst_4, 1000)
print('\n')
OTM_decresce(qnt_entradas, Inst_4, 1000)

A melhor solução encontrada foi 95
Tempo de execução com k = 10 : 0.04409337043762207 segundos


A melhor solução encontrada foi 103
Tempo de execução: 0.003972768783569336 segundos


In [ ]:
with open('Hard28_BPP178.txt', 'r') as f:
    results = [int(line) for line in f.readlines()]
    #print(results)

qnt_entradas = results[0]
results.remove(results[0])
results.remove(1000)
#print(results)
Inst_5 = results

OTM_shuffle(qnt_entradas, Inst_5, 1000)
print('\n')
OTM_decresce(qnt_entradas, Inst_5, 1000)

A melhor solução encontrada foi 106
Tempo de execução com k = 10 : 0.0444636344909668 segundos


A melhor solução encontrada foi 113
Tempo de execução: 0.003900766372680664 segundos


In [ ]:
with open('Hard28_BPP14.txt', 'r') as f:
    results = [int(line) for line in f.readlines()]
    #print(results)

qnt_entradas = results[0]
results.remove(results[0])
results.remove(1000)
#print(results)
Inst_6 = results

OTM_shuffle(qnt_entradas, Inst_6, 1000)
print('\n')
OTM_decresce(qnt_entradas, Inst_6, 1000)

A melhor solução encontrada foi 78
Tempo de execução com k = 10 : 0.03149151802062988 segundos


A melhor solução encontrada foi 86
Tempo de execução: 0.0038788318634033203 segundos


In [ ]:
with open('Hard28_BPP47.txt', 'r') as f:
    results = [int(line) for line in f.readlines()]
    #print(results)

qnt_entradas = results[0]
results.remove(results[0])
results.remove(1000)
#print(results)
Inst_7 = results

OTM_shuffle(qnt_entradas, Inst_7, 1000)
print('\n')
OTM_decresce(qnt_entradas, Inst_7, 1000)

A melhor solução encontrada foi 95
Tempo de execução com k = 10 : 0.0364995002746582 segundos


A melhor solução encontrada foi 98
Tempo de execução: 0.0031371116638183594 segundos


In [ ]:
with open('Hard28_BPP119.txt', 'r') as f:
    results = [int(line) for line in f.readlines()]
    #print(results)

qnt_entradas = results[0]
results.remove(results[0])
results.remove(1000)
#print(results)
Inst_8 = results

OTM_shuffle(qnt_entradas, Inst_8, 1000)
print('\n')
OTM_decresce(qnt_entradas, Inst_8, 1000)

A melhor solução encontrada foi 100
Tempo de execução com k = 10 : 0.04345989227294922 segundos


A melhor solução encontrada foi 107
Tempo de execução: 0.0039174556732177734 segundos


In [ ]:
with open('Hard28_BPP175.txt', 'r') as f:
    results = [int(line) for line in f.readlines()]
    #print(results)

qnt_entradas = results[0]
results.remove(results[0])
results.remove(1000)
#print(results)
Inst_9 = results

OTM_shuffle(qnt_entradas, Inst_9, 1000)
print('\n')
OTM_decresce(qnt_entradas, Inst_9, 1000)

A melhor solução encontrada foi 111
Tempo de execução com k = 10 : 0.04486870765686035 segundos


A melhor solução encontrada foi 116
Tempo de execução: 0.003928184509277344 segundos


In [ ]:
with open('Hard28_BPP181.txt', 'r') as f:
    results = [int(line) for line in f.readlines()]
    #print(results)

qnt_entradas = results[0]
results.remove(results[0])
results.remove(1000)
#print(results)
Inst_10 = results

OTM_shuffle(qnt_entradas, Inst_10, 1000)
print('\n')
OTM_decresce(qnt_entradas, Inst_10, 1000)

A melhor solução encontrada foi 95
Tempo de execução com k = 10 : 0.037174224853515625 segundos


A melhor solução encontrada foi 100
Tempo de execução: 0.0031697750091552734 segundos
